In [4]:
# Restart kernel and reimport everything to ensure fresh imports
import importlib
import sys

# Clear any cached modules
if 'grand_file' in sys.modules:
    del sys.modules['grand_file']
if 'geolib' in sys.modules:
    del sys.modules['geolib']

import ee
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image, display
import json
sys.path.append('../Setup')  # Go up one level to reach Setup folder
from gee_setup import *

setup_gee()

# Import with fresh reload
from grand_file import snow_difference_map

✅ Matplotlib configured
Google Earth Engine initialized successfully!
EE version: 1.6.3
✅ Matplotlib configured
Google Earth Engine initialized successfully!
EE version: 1.6.3
✅ Matplotlib configured
Google Earth Engine initialized successfully!
EE version: 1.6.3
✅ Matplotlib configured
Google Earth Engine initialized successfully!
EE version: 1.6.3
✅ Matplotlib configured
Google Earth Engine initialized successfully!
EE version: 1.6.3
Google Earth Engine initialized successfully!
EE version: 1.6.3


In [2]:
ee.Authenticate()

True

In [18]:
# Original Rocky Mountains area polygon
rocky_aoi = ee.Geometry.Polygon([[
    [-106.99065650552905, 39.088354116315266],
    [-106.71050513834155, 38.652122067998306], 
    [-106.14470923990405, 38.596331654032184],
    [-106.40838111490405, 39.21189167999414],
    [-106.99065650552905, 39.088354116315266]
]])

# Generate new raw and visualized maps
snow_difference_map(rocky_aoi, 
                   output_filename='rocky_mountains_adaptive', 
                   output_format='tiff',
                   cloud_cover=20)

Area: 0.5 sq deg, Resolution: 30m
Raw data exported: outputs\raw_data\rocky_mountains_adaptive_historical_raw.tiff (30m)
Raw data exported: outputs\raw_data\rocky_mountains_adaptive_historical_raw.tiff (30m)
Visualized exported: outputs\visualized\rocky_mountains_adaptive_historical_visualized.tiff (60m)
Visualized exported: outputs\visualized\rocky_mountains_adaptive_historical_visualized.tiff (60m)
Raw data exported: outputs\raw_data\rocky_mountains_adaptive_recent_raw.tiff (30m)
Raw data exported: outputs\raw_data\rocky_mountains_adaptive_recent_raw.tiff (30m)
Visualized exported: outputs\visualized\rocky_mountains_adaptive_recent_visualized.tiff (60m)
Visualized exported: outputs\visualized\rocky_mountains_adaptive_recent_visualized.tiff (60m)
Raw data exported: outputs\raw_data\rocky_mountains_adaptive_difference_raw.tiff (30m)
Raw data exported: outputs\raw_data\rocky_mountains_adaptive_difference_raw.tiff (30m)
Visualized exported: outputs\visualized\rocky_mountains_adaptive_dif

In [15]:
## summary statistics analysis on .tiff files
def analyze_snowpack_tiff(tiff_path, filename, difference_map_bool=False):
    import rasterio
    import numpy as np
    with rasterio.open(tiff_path) as raster:
        data = raster.read(1) ## read the first band (should be NDSI index)

        # mask no data values
        masked_data = np.ma.masked_equal(data, raster.nodata)
        
        if len(masked_data) == 0:
            print("No valid data found in the raster.")
            return None
        stats = pd.DataFrame({
            "filename": filename,
            "mean": masked_data.mean(),
            "median": np.ma.median(masked_data),
            "min_value": masked_data.min(),
            "max_value": masked_data.max(),
            "std_dev": masked_data.std(),
            "total_pixels": masked_data.count(),
            "snow pixels": np.sum(masked_data > 0.4) if difference_map_bool is False else None,
            "snow_coverage_percentage": (np.sum(masked_data > 0.4) / masked_data.count()) * 100 if difference_map_bool is False else None
        }, index=[0])

        if difference_map_bool is True:
            difference_map_stats = pd.DataFrame({
                "mean_difference": masked_data.mean(),
                "median_difference": np.ma.median(masked_data),
                "mild snow loss pixels": np.sum((masked_data < -0.1) & (masked_data >= -0.3)),
                "severe snow loss pixels": np.sum(masked_data < -0.3),
                "total_loss_pixels": np.sum(masked_data < -0.1),
                "total_gain_pixels": np.sum(masked_data > 0.1),
                "percentage_snow_loss": (np.sum(masked_data < -0.1) / masked_data.count()) * 100,
                "percentage_severe_snow_loss": (np.sum(masked_data < -0.3) / masked_data.count()) * 100
            }, index=[0])
            appended_stats = pd.concat([stats, difference_map_stats], axis=1)
            return appended_stats
        return stats

In [24]:
# Now analyze the raw difference data (actual NDSI values)
difference_raw = analyze_snowpack_tiff('outputs/raw_data/rocky_mountains_adaptive_difference_raw.tiff', 'Rocky_Mountains_Raw_Diff', difference_map_bool=True)
print("RAW NDSI DIFFERENCE ANALYSIS:")
print(difference_raw)
difference_raw.to_csv('outputs/rocky_mountains_adaptive_raw_difference_analysis.csv', index=False)

RAW NDSI DIFFERENCE ANALYSIS:
                   filename      mean    median  min_value  max_value  \
0  Rocky_Mountains_Raw_Diff -0.061463 -0.038696  -1.033178   0.847858   

    std_dev  total_pixels snow pixels snow_coverage_percentage  \
0  0.089247       9206739        None                     None   

   mean_difference  median_difference  mild snow loss pixels  \
0        -0.061463          -0.038696                1807056   

   severe snow loss pixels  total_loss_pixels  total_gain_pixels  \
0                   229784            2036840             116526   

   percentage_snow_loss  percentage_severe_snow_loss  
0              22.12336                     2.495824  


In [21]:
# The above results show RGB values (1-255), not NDSI values (-1 to +1)
# Let's try the raw data version if it exists
import os
if os.path.exists('outputs/raw_data/rocky_mountains_adaptive_difference_raw.tiff'):
    print("Found raw data version!")
    difference_raw = analyze_snowpack_tiff('outputs/raw_data/rocky_mountains_adaptive_difference_raw.tiff', 'Rocky_Mountains_Raw_Diff', difference_map_bool=True)
    print(difference_raw)
else:
    print("No raw data version found. The analysis above used RGB/visualized data.")

Found raw data version!
                   filename      mean    median  min_value  max_value  \
0  Rocky_Mountains_Raw_Diff -0.061463 -0.038696  -1.033178   0.847858   

    std_dev  total_pixels snow pixels snow_coverage_percentage  \
0  0.089247       9206739        None                     None   

   mean_difference  median_difference  mild snow loss pixels  \
0        -0.061463          -0.038696                1807056   

   severe snow loss pixels  total_loss_pixels  total_gain_pixels  \
0                   229784            2036840             116526   

   percentage_snow_loss  percentage_severe_snow_loss  
0              22.12336                     2.495824  
                   filename      mean    median  min_value  max_value  \
0  Rocky_Mountains_Raw_Diff -0.061463 -0.038696  -1.033178   0.847858   

    std_dev  total_pixels snow pixels snow_coverage_percentage  \
0  0.089247       9206739        None                     None   

   mean_difference  median_difference  m